In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
URL = 'https://egb.com/esports/counter-strike'
soup = BeautifulSoup(requests.get(URL).content, 'html.parser')

In [3]:
# for data in soup.html.body:
#     print(data)
#     print(20*'-')  table-bets table-bets--full table-bets--compact
soup.findAll("div", {"class": "table-bets table-bets--full"})

[]

In [4]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') #executable_path=yourdriver
driver.get(URL)
html = driver.page_source

In [5]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [6]:
table = driver.find_elements_by_class_name('table-bets')
table = table[0].text

In [9]:
table

'DATE (UTC)\nEVENT\nPLAYER\nODDS\nPLAYER\nODDS\n18.07\n19:55\nIEM\nVitality\n1.173\nHeroic\n3.748\n17.07\n15:15\nEU Minor\nFnatic\n1.306\nAncient\n2.929\n17.07\n15:15\nEU Minor\nCR4ZY\n1.344\nSprout\n2.776\n17.07\n15:00\nKOB\nCopenhagen Flames\n1.496\nSyman\n2.352\n17.07\n14:00\nEU Minor\nNorth\n1.329\nBIG\n2.833\n17.07\n14:00\nEU Minor\nmousesports\n1.147\nNoChance\n4.000\n17.07\n09:15\nNA Minor\neUnited\n1.510\nLuminosity\n2.323\n17.07\n09:15\nNA Minor\nSingularity\n1.598\nSharks\n2.166\n17.07\n08:00\nNA Minor\nFURIA\n1.190\nINTZ\n3.607\n17.07\n08:00\nNA Minor\nNRG\n1.146\nTeamOne\n4.010\n17.07\n01:00\nESEA NA\nGood Game PR\n1.496\nDogs of War\n2.352\n17.07\n01:00\nESEA NA\nJust Swing\n1.859\nFront\n1.861\n17.07\n01:00\nESEA NA\nBig Frames\n1.470\nLiviD\n2.410\n17.07\n00:00\nESEA Brasil\nDETONA\n1.642\npaiN\n2.100\n16.07\n23:30\nESEA Brasil\nKeyd\n1.559\nReapers\n2.230\n16.07\n22:00\nBrasil Game Cup\nDETONA\n1.885\nIsurus\n1.832\n16.07\n21:00\nBrasil Game Cup\nIsurus\n1.225\nRED Cani

In [11]:
def idx_time_regex_search(row):
    """Get the index of the first occurrence of a time in xx:xx format."""

    res_idx = -1
    for idx, token in enumerate(row):
        if re.search(r'^(\d{1,2}(:\d{1,2}))$', token):
            res_idx = idx
            break

    return res_idx

In [147]:
def insert_row_breaks(table, break_token='ROW_BREAK'):
    processed_table = []
    insert_counter = 0
    for tok_idx, token in enumerate(table):
        if re.search(r'^(\d{1,2}(:\d{1,2}))$', token):
            processed_table.insert(tok_idx + insert_counter - 1, break_token)
            insert_counter += 1
        elif token == 'Live!':
            processed_table.insert(tok_idx + insert_counter, break_token)
            insert_counter += 1
        processed_table.append(token)
    if processed_table[0] == break_token:
        processed_table.pop(0)
    return processed_table

def reformat_table(table, break_token='ROW_BREAK'):
    processed_table, row = [], []
    for token in table:
        if token == break_token:
            processed_table.append(row)
            row = []
            continue
        row.append(token)
    return processed_table

import time
import datetime

def get_match_time(tm, dt):
    """Extract the match time from a row od data."""

    try:  # TODO: implement robust year filling method
        match_time = datetime.datetime.strptime(dt + ' ' + tm, '%d.%m %H:%M').replace(year=datetime.datetime.now().year)
        match_time = int(datetime.datetime.timestamp(match_time))
    except:
        match_time = -1

    return match_time

def string_to_float(s):
    """Convert string s is a number and returns boolean."""

    try:
        return float(s)
    except ValueError:
        return -1

def reformat_row(row):
    dt, tm, tournament_name, contestant_1, contestant_1_odds, contestant_2, contestant_2_odds = row
    contestant_1_odds = string_to_float(contestant_1_odds)
    contestant_2_odds = string_to_float(contestant_2_odds)
    match_time = get_match_time(tm, dt)
    scrape_time = int(time.time())
    source, bet_type, draw_odds = 'egb', 'winner', -1
    return (contestant_1, contestant_2, contestant_1_odds, contestant_2_odds, draw_odds, bet_type, scrape_time, match_time, tournament_name, source)



In [148]:
rows = table.split('\n')
row_data = rows[6:]
processed_table = reformat_table(insert_row_breaks(row_data))
db_table = [reformat_row(row) for row in processed_table if len(row) == 7]

pd.DataFrame(db_table)

,0,1,2,3,4,5,6,7,8,9
0,Vitality,Heroic,1.173,3.748,-1,winner,1563313686,1563504900,IEM,egb
1,Fnatic,Ancient,1.306,2.929,-1,winner,1563313686,1563401700,EU Minor,egb
2,CR4ZY,Sprout,1.344,2.776,-1,winner,1563313686,1563401700,EU Minor,egb
3,Copenhagen Flames,Syman,1.496,2.352,-1,winner,1563313686,1563400800,KOB,egb
4,North,BIG,1.329,2.833,-1,winner,1563313686,1563397200,EU Minor,egb
5,mousesports,NoChance,1.147,4.000,-1,winner,1563313686,1563397200,EU Minor,egb
6,eUnited,Luminosity,1.510,2.323,-1,winner,1563313686,1563380100,NA Minor,egb
7,Singularity,Sharks,1.598,2.166,-1,winner,1563313686,1563380100,NA Minor,egb
8,FURIA,INTZ,1.190,3.607,-1,winner,1563313686,1563375600,NA Minor,egb
9,NRG,TeamOne,1.146,4.010,-1,winner,1563313686,1563375600,NA Minor,egb


In [51]:
len(rows)

268

In [52]:
nprows = np.array(rows[6:216])
nprows = nprows.reshape((-1,7))
nprows

array([['18.07', '19:55', 'IEM', 'Vitality', '1.173', 'Heroic', '3.748'],
       ['17.07', '15:15', 'EU Minor', 'Fnatic', '1.306', 'Ancient',
        '2.929'],
       ['17.07', '15:15', 'EU Minor', 'CR4ZY', '1.344', 'Sprout',
        '2.776'],
       ['17.07', '15:00', 'KOB', 'Copenhagen Flames', '1.496', 'Syman',
        '2.352'],
       ['17.07', '14:00', 'EU Minor', 'North', '1.329', 'BIG', '2.833'],
       ['17.07', '14:00', 'EU Minor', 'mousesports', '1.147', 'NoChance',
        '4.000'],
       ['17.07', '09:15', 'NA Minor', 'eUnited', '1.510', 'Luminosity',
        '2.323'],
       ['17.07', '09:15', 'NA Minor', 'Singularity', '1.598', 'Sharks',
        '2.166'],
       ['17.07', '08:00', 'NA Minor', 'FURIA', '1.190', 'INTZ', '3.607'],
       ['17.07', '08:00', 'NA Minor', 'NRG', '1.146', 'TeamOne', '4.010'],
       ['17.07', '01:00', 'ESEA NA', 'Good Game PR', '1.496',
        'Dogs of War', '2.352'],
       ['17.07', '01:00', 'ESEA NA', 'Just Swing', '1.859', 'Front',
        

['18.07',
 '19:55',
 'IEM',
 'Vitality',
 '1.173',
 'Heroic',
 '3.748',
 '17.07',
 '15:15',
 'EU Minor',
 'Fnatic',
 '1.306',
 'Ancient',
 '2.929',
 '17.07',
 '15:15',
 'EU Minor',
 'CR4ZY',
 '1.344',
 'Sprout',
 '2.776',
 '17.07',
 '15:00',
 'KOB',
 'Copenhagen Flames',
 '1.496',
 'Syman',
 '2.352',
 '17.07',
 '14:00',
 'EU Minor',
 'North',
 '1.329',
 'BIG',
 '2.833',
 '17.07',
 '14:00',
 'EU Minor',
 'mousesports',
 '1.147',
 'NoChance',
 '4.000',
 '17.07',
 '09:15',
 'NA Minor',
 'eUnited',
 '1.510',
 'Luminosity',
 '2.323',
 '17.07',
 '09:15',
 'NA Minor',
 'Singularity',
 '1.598']

In [7]:
soup = BeautifulSoup(table, 'html.parser')

In [10]:
table == soup.text

True

In [157]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def remove_header(html, cutoff='RESULTS', separator='\n', padding_token='_PADDING_'):
    tokenized_html = html.split(separator)
    for idx, token in enumerate(tokenized_html):
        if token == cutoff:
            break
    headless_html = padding_token.join(tokenized_html[idx+1:])
    return headless_html
    
def insert_row_breaks(html, separator='_PADDING_', row_break_token='_ROW_BREAK_', padding_token='_PADDING_'):
    tokenized_html = html.split(separator)
    num_tokens = len(tokenized_html)
    insert = False
    processed_html = [tokenized_html[0]]
    for idx in range(1, num_tokens):        
        if insert and not is_number(tokenized_html[idx]):
            insert = False
            processed_html.append(row_break_token)
        elif tokenized_html[idx - 1] == 'Over' and tokenized_html[idx] == 'Under':
            insert = True
            continue
        processed_html.append(tokenized_html[idx])
    processed_html = padding_token.join(processed_html)
    return processed_html

def idx_search(l, s):
    for idx, token in enumerate(l):
        if s in token:
            break
    return idx

def encode_row(row, separator='_PADDING_', stopwords={'_', 'LIVE', '', '-'}):
    # TODO: NA is previous tournament, means no colkumn headers
    
    result = {}
    row = row.split(separator)
    
    time_idx = idx_search(row, ':')
            
    # tournament name
    if len(set(row[:time_idx]) - stopwords) == 1:
        result['tournament_name'] = (set(row[:time_idx]) - stopwords).pop()
    elif len(set(row[:time_idx]) - stopwords) > 1:
        print(set(row[:time_idx]) - stopwords)
        result['tournament_name'] = 'NA'
    else:
        result['tournament_name'] = 'NA'
    
    # time and date
    result['time'] = row[time_idx]
    result['date'] = row[time_idx + 1]
    
    row = row[time_idx + 2:]
    winner_idx = idx_search(row, 'WINNER')
    if 'Total' in row[:winner_idx]:  # bad fit -> correct
        contestants = []
        for winner_idx, token in enumerate(row):
            if is_number(token):
                winner_idx += 1
                break
            contestants.append(token)
        result['contestants'] = contestants
    else:  
        result['contestants'] = row[:winner_idx]
    
    row = row[winner_idx+1:]
    
    # winner odds
    result['winner_odds'] = row[:2]
    
    row = row[2:]
    
    # round handicap
    result['handicap_type'] = row[0]
    result['handicap_odds'] = row[1:5]
    
    row = row[5:]
    
    # map bet
    result['map_type'] = row[0]
    result['map_total'] = row[3]
    result['map_over_odds'] = row[4]
    result['map_under_odds'] = row[5]

    row = row[6:]
    
    print(row)
    
    return result

In [161]:
text = remove_header(soup.text)
text = insert_row_breaks(text)
rows = text.split('_ROW_BREAK_')
for row in rows:
    result = encode_row(row)
    print(result)
#     break

['19', '']
{'tournament_name': 'GG.BET COLOGNE INVITATIONAL', 'time': '05:00', 'date': 'JUN 13', 'contestants': ['Sprout', 'X', 'NoChance'], 'winner_odds': ['1.6', '2.23'], 'handicap_type': '1ST MAP - ROUND HANDICAP (INCL. OVERTIME)', 'handicap_odds': ['-3.5', '2.1', '+3.5', '1.67'], 'map_type': '1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)', 'map_total': '26.5', 'map_over_odds': '1.9', 'map_under_odds': '1.83'}
['19', '']
{'tournament_name': 'GAMEAGENTS LEAGUE SEASON 3', 'time': '05:00', 'date': 'JUN 13', 'contestants': ['Nemiga', 'X', 'Nordavind'], 'winner_odds': ['1.98', '1.76'], 'handicap_type': '1ST MAP - ROUND HANDICAP (INCL. OVERTIME)', 'handicap_odds': ['+3.5', '1.54', '-3.5', '2.35'], 'map_type': '1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)', 'map_total': '26.5', 'map_over_odds': '1.88', 'map_under_odds': '1.85'}
['45', '']
{'tournament_name': 'LOOT.BET SMACK MY BEACH CIS QUALIFIER', 'time': '06:00', 'date': 'JUN 13', 'contestants': ['Syman', 'X', 'NONAME'], 'winner_odds': ['1.1', '6.1

IndexError: list index out of range

In [162]:
row

'_PADDING_LIVE_PADDING_07:00_PADDING_JUN 13_PADDING_AGO_PADDING_X_PADDING_ALTERNATE aTTaX_PADDING_1.59_PADDING_2.25_PADDING_-3.5_PADDING_2.09_PADDING_+3.5_PADDING_1.68_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.9_PADDING_1.83_PADDING_19_PADDING_'

In [160]:
len(rows)

36

In [156]:
text

"GG.BET COLOGNE INVITATIONAL_PADDING_LIVE_PADDING_05:00_PADDING_JUN 13_PADDING_Sprout_PADDING_X_PADDING_NoChance_PADDING_WINNER_PADDING_1.6_PADDING_2.23_PADDING_1ST MAP - ROUND HANDICAP (INCL. OVERTIME)_PADDING_-3.5_PADDING_2.1_PADDING_+3.5_PADDING_1.67_PADDING_1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.9_PADDING_1.83_PADDING_19_PADDING__ROW_BREAK__PADDING_GAMEAGENTS LEAGUE SEASON 3_PADDING_LIVE_PADDING_05:00_PADDING_JUN 13_PADDING_Nemiga_PADDING_X_PADDING_Nordavind_PADDING_WINNER_PADDING_1.98_PADDING_1.76_PADDING_1ST MAP - ROUND HANDICAP (INCL. OVERTIME)_PADDING_+3.5_PADDING_1.54_PADDING_-3.5_PADDING_2.35_PADDING_1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.88_PADDING_1.85_PADDING_19_PADDING__ROW_BREAK__PADDING_LOOT.BET SMACK MY BEACH CIS QUALIFIER_PADDING_LIVE_PADDING_06:00_PADDING_JUN 13_PADDING_Syman_PADDING_X_PADDING_NONAME_PADDING_WINNER_PADDING_1.1_PADDING_6.19_PADDING_MAP HANDICAP_PADDIN

In [65]:
text.split('ROW_BREAK')[0]

'FILTERS_PADDING_MATCH_PADDING_OF THE DAY_PADDING_WINNER_PADDING_Heroic_PADDING_2.15_PADDING_AVANGAR_PADDING_1.65_PADDING_+67_PADDING_UPCOMING GAMES_PADDING_OUTRIGHTS_PADDING_RESULTS_PADDING_GG.BET COLOGNE INVITATIONAL_PADDING_LIVE_PADDING_05:00_PADDING_JUN 13_PADDING_Sprout_PADDING_X_PADDING_NoChance_PADDING_WINNER_PADDING_1.6_PADDING_2.23_PADDING_1ST MAP - ROUND HANDICAP (INCL. OVERTIME)_PADDING_-3.5_PADDING_2.1_PADDING_+3.5_PADDING_1.67_PADDING_1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.9_PADDING_1.83_PADDING_19_PADDING__'

In [19]:
soup.text

"FILTERS\nMATCH\nOF THE DAY\nWINNER\nHeroic\n2.15\nAVANGAR\n1.65\n+67\nUPCOMING GAMES\nOUTRIGHTS\nRESULTS\nGG.BET COLOGNE INVITATIONAL\nLIVE\n05:00\nJUN 13\nSprout\nX\nNoChance\nWINNER\n1.6\n2.23\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n-3.5\n2.1\n+3.5\n1.67\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n26.5\n1.9\n1.83\n19\nGAMEAGENTS LEAGUE SEASON 3\nLIVE\n05:00\nJUN 13\nNemiga\nX\nNordavind\nWINNER\n1.98\n1.76\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n+3.5\n1.54\n-3.5\n2.35\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n26.5\n1.88\n1.85\n19\nLOOT.BET SMACK MY BEACH CIS QUALIFIER\nLIVE\n06:00\nJUN 13\nSyman\nX\nNONAME\nWINNER\n1.1\n6.19\nMAP HANDICAP\n-1.5\n1.42\n+1.5\n2.71\nTOTAL MAPS\nTotal\nOver\nUnder\n2.5\n2.98\n1.36\n45\nGG.BET COLOGNE INVITATIONAL\nLIVE\n06:15\nJUN 13\nLiquid\nX\nSpirit\nWINNER\n1.12\n5.61\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n-7.5\n1.92\n+7.5\n1.81\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n24.5\n1.9